# Competitive Auctions on eBay.com

## eBay.com의 경쟁적인 경매

- [eBayAuction.csv] 파일에는 2004년 5~6월에 eBay.com에서 거래된 총 1,972건의 경매 정보가 담겨 있다. 분석 목적은 이 데이터를 사용해 경쟁적인 경매와 비경쟁적인 경매를 구분하는 모델을 구축하는 것이다.

- 경쟁적인 경매는 경매되고 있는 물품에 대해 최소한 2개 이상의 입찰이 있는 경매로 정의된다. 데이터는 물품을 나타내는 변수(경매 범주), 판매자 등급(eBay 등급), 판매자가 선택한 경매 조건(경매 기간, 시작가(OpenPrice), 화폐 단위, 경매 마감 요일)을 포함한다. 경매가 마감된 가격(ClosePrice)에 대한 데이터도 있다. 분석 목표는 경매가 경쟁적인지 아닌지를 예측하는 것이다.

- 데이터 전처리: 범주형 예측 변수들에 대해서 더미 변수를 생성하시오. 범주형 변수에는 물품 항목(Category: 18개 범주), 화폐 단위(currency: US, GBP, EUR), 경매 종료일(endDay: 월요일~일요일), 경매 기간(Duration: 1, 3, 5, 7, 10일)이 포함된다.

In [1]:
'''
eBayAuctions.csv파일을 읽어 DataFrame 생성
'''
import pandas as pd

eBayAuctions_df = pd.read_csv('../data/eBayAuctions.csv')
eBayAuctions_df


,Category,currency,sellerRating,Duration,endDay,ClosePrice,OpenPrice,Competitive?
0,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
1,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
2,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
3,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
4,Music/Movie/Game,US,3249,5,Mon,0.01,0.01,0
...,...,...,...,...,...,...,...,...
1967,Automotive,US,2992,5,Sun,359.95,359.95,0
1968,Automotive,US,21,5,Sat,610.00,300.00,1
1969,Automotive,US,1400,5,Mon,549.00,549.00,0
1970,Automotive,US,57,7,Fri,820.00,650.00,1


# 데이터 분석

## 히스토그램 분석

In [2]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import dmba
import numpy as np

num_col = 3
num_row = 3
fig = plt.figure(figsize=(15, 15))
gs = GridSpec(num_col, num_row, figure=fig)

fig.suptitle("Histogram of Variables", fontsize=20)

bins_dict = {}
for idx, col in enumerate(eBayAuctions_df.columns):

    ax = fig.add_subplot(gs[idx//num_col, idx%num_row])
    ax.set_title(f"{col}")
        
    if col in ['sellerRating', 'ClosePrice', 'OpenPrice']:  # 분포가 큰 columns
        bins = 20
        q1 = eBayAuctions_df[col].quantile(0.1) # 하위 10% 값
        q3 = eBayAuctions_df[col].quantile(0.9) # 상위 90% 값
        extracted_df = eBayAuctions_df[(eBayAuctions_df[col] >= q1) & (eBayAuctions_df[col] <= q3)] # 10~90%만 추출

    else:  # 분포가 작은 columns
        bins = len(set(eBayAuctions_df[col]))
        extracted_df = eBayAuctions_df

    counts, bins, patches = ax.hist(extracted_df[col].dropna(), bins=bins)
    bins_dict[col] = bins

    # x축 라벨 형식 변경
    if col in ['Category']:
        ax.set_xticklabels(set(extracted_df[col]), rotation=45, ha='right')

    elif col in ['endDay']:
        ax.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

    for count, patch in zip(counts, patches):   # 각 bin의 count 수 표시
        x = patch.get_x() + patch.get_width() / 2
        y = count
        ax.text(x, y, f"{int(count)}", ha='center', va='bottom', fontsize=10, color='black')

fig.tight_layout()

no display found. Using non-interactive Agg backend


/tmp/ipykernel_1650233/3698312791.py:34: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(set(extracted_df[col]), rotation=45, ha='right')
/tmp/ipykernel_1650233/3698312791.py:37: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])


## 변수 별 각 카테고리의 평균 라벨 수치

In [3]:
outcome = 'Competitive?'

def createGraph(group, col, row):
    groupAverage = eBayAuctions_df.groupby([group])[outcome].mean()
    if group == 'endDay': # rotate so that display starts on Sunday
        groupAverage = groupAverage.reindex(index=np.roll(groupAverage.index,1))
        groupAverage.index = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']

    ax = fig.add_subplot(gs[col, row])
    groupAverage.plot.bar(color='C0', ax=ax)
    ax.set_title(f"{group}")
    ax.set_ylabel('Average Competitive')
    if group == 'Category':
        ax.set_xticklabels(groupAverage.index, rotation=45, ha='right')
    else:
        ax.set_xticklabels(groupAverage.index, rotation=0, ha='right')
    return ax

def graphNumericalGraph(group, col, row):
    q1 = eBayAuctions_df[group].quantile(0.1) # 하위 10% 값
    q3 = eBayAuctions_df[group].quantile(0.9) # 상위 90% 값
    extracted_df = eBayAuctions_df[(eBayAuctions_df[group] >= q1) & (eBayAuctions_df[group] <= q3)] # 10~90%만 추출
    temp_df = pd.DataFrame({group:  pd.cut(extracted_df[group], bins=bins_dict[group], include_lowest=True),
                            outcome: extracted_df[outcome]})
    groupAverage = temp_df.groupby([group])[outcome].mean()

    ax = fig.add_subplot(gs[col, row])
    groupAverage.plot.bar(color='C0', ax=ax)
    ax.set_title(f"{group}")
    ax.set_ylabel('Average Competitive')
    ax.set_xticklabels(groupAverage.index, rotation=45, ha='right')

num_col = 3
num_row = 3
fig = plt.figure(figsize=(15, 15))
gs = GridSpec(num_col, num_row, figure=fig)

fig.suptitle("Average Competitive of Variables", fontsize=20)

createGraph('Category', 0, 0)
createGraph('currency', 0, 1)
graphNumericalGraph('sellerRating', 0, 2)
createGraph('Duration', 1, 0)
createGraph('endDay', 1, 1)
graphNumericalGraph('ClosePrice', 1, 2)
graphNumericalGraph('OpenPrice', 2, 1)
plt.tight_layout()

plt.show()

/tmp/ipykernel_1650233/235711962.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupAverage = temp_df.groupby([group])[outcome].mean()
/tmp/ipykernel_1650233/235711962.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupAverage = temp_df.groupby([group])[outcome].mean()
/tmp/ipykernel_1650233/235711962.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupAverage = temp_df.groupby([group])[outcome].mean(

* Category : 'Antique/Art/Craft'는 가장 경쟁적인 항목이며, 'Coins/Stamps'는 가장 덜 경쟁적인 항목이다.
* currency : 'GBP'는 가장 경쟁적인 통화이며, 'EUR'는 가장 덜 경쟁적인 통화이다.
* Duration : '5일'은 가장 경쟁적인 기간이며, '1일'은 가장 덜 경쟁적인 기간이다.
* endDay   : 'Mon'는 가장 경쟁적인 마감일이며, 'Wed'는 가장 덜 경쟁적인 마감일이다.


---
- 다양한 범주형 변수들(더미 변수가 아닌 원래 변수 사용)의 함수로서 이진형 결과 변수(경쟁적인 경매 여부)의 평균을 구하기 위해 피벗 테이블을 작성하시오. 모델에서 사용될 더미 변수의 개수를 줄이기 위해서 피벗 테이블의 정보를 활용하시오. 예를 들어, 경쟁적인 경매의 분포가 유사한 범주는 합칠 수 있다.

## Category(카테고리)
[eBay.com - Category](https://www.ebay.com/n/all-categories)

- Antique/Art/Craft      앤틱/미술/공예   
- Automotive             자동차   
- Books                  책   
- Business/Industrial    비즈니스/산업   
- Clothing/Accessories   옷/악세사리   
- Coins/Stamps           동전/우표   
- Collectibles           수집물   
- Computer               컴퓨터   
- Electronics            전자기기   
- EverythingElse         이외의 것들   
- Health/Beauty          헬스/뷰티   
- Home/Garden            홈/가든   
- Jewelry                쥬얼리   
- Music/Movie/Game       음악/영화/게임   
- Photography            사진   
- Pottery/Glass          도자기/글라스   
- SportingGoods          운동용품   
- Toys/Hobbies           장난감/취미   

In [4]:
# 시각화 (seaborn 라이브러리 사용)
import seaborn as sns
import matplotlib.pyplot as plt

Category_pivot = eBayAuctions_df.pivot_table(index='Category', values='Competitive?', aggfunc='mean')
print(Category_pivot)
# heatmap 시각화
sns.heatmap(Category_pivot, cmap='coolwarm')
plt.show()

                      Competitive?
Category                          
Antique/Art/Craft         0.564972
Automotive                0.353933
Books                     0.500000
Business/Industrial       0.666667
Clothing/Accessories      0.504202
Coins/Stamps              0.297297
Collectibles              0.577406
Computer                  0.666667
Electronics               0.800000
EverythingElse            0.235294
Health/Beauty             0.171875
Home/Garden               0.656863
Jewelry                   0.365854
Music/Movie/Game          0.602978
Photography               0.846154
Pottery/Glass             0.350000
SportingGoods             0.725806
Toys/Hobbies              0.529915


In [5]:
import pandas as pd
import numpy as np

# 데이터프레임 생성 (샘플 데이터)
eBayAuctions_df

# 범주별 평균값 계산
category_means = eBayAuctions_df.groupby("Category")["Competitive?"].mean()

# 오차 범위 내 병합을 위한 함수
def merge_categories(category_means, tolerance=0.05):
    category_means = category_means.sort_values()  # 오름차순 정렬
    groups = []  # 병합 결과 저장
    current_group = []
    current_mean = None

    for category, mean in category_means.items():
        if current_group and abs(current_mean - mean) > tolerance:
            groups.append(current_group)
            current_group = []
        current_group.append(category)
        current_mean = mean
    if current_group:
        groups.append(current_group)

    # 병합된 그룹 반환 (카테고리 이름들을 "-"로 연결)
    return {cat: "+".join(group) for group in groups for cat in group}

# 병합된 그룹 생성
merged_groups = merge_categories(category_means)

# 데이터프레임에 병합된 그룹 추가
eBayAuctions_df["MergedCategory"] = eBayAuctions_df["Category"].map(merged_groups)

merged_Category_pivot = eBayAuctions_df.pivot_table(index='MergedCategory', 
                                       values='Competitive?', 
                                       aggfunc='mean')
print(merged_Category_pivot)
# heatmap 시각화
sns.heatmap(merged_Category_pivot, cmap='coolwarm')
plt.show()

                                                    Competitive?
MergedCategory                                                  
Books+Clothing/Accessories+Toys/Hobbies+Antique...      0.564437
Coins/Stamps                                            0.297297
Electronics+Photography                                 0.808824
EverythingElse                                          0.235294
Health/Beauty                                           0.171875
Home/Garden+Computer+Business/Industrial                0.660256
Pottery/Glass+Automotive+Jewelry                        0.357143
SportingGoods                                           0.725806


---
# currency(통화)
- EUR: 유로
- GBP: 파운드 스털링(영국 외 9개 국가)
- US: 달러

In [6]:
currency_pivot = eBayAuctions_df.pivot_table(index='currency', values='Competitive?', aggfunc='mean')
print(currency_pivot)
# heatmap 시각화
sns.heatmap(currency_pivot, cmap='coolwarm')
plt.show()

          Competitive?
currency              
EUR           0.551595
GBP           0.687075
US            0.519350


---
## endDay(경매 종료일)
- Monday
- Tuesday
- Wednsday
- Thursday
- Friday
- Saturday
- Sunday

In [7]:
endDay_pivot = eBayAuctions_df.pivot_table(index='endDay', values='Competitive?', aggfunc='mean')
print(endDay_pivot)
# heatmap 시각화
sns.heatmap(endDay_pivot, cmap='coolwarm')
plt.show()

        Competitive?
endDay              
Fri         0.466899
Mon         0.673358
Sat         0.427350
Sun         0.485207
Thu         0.603960
Tue         0.532164
Wed         0.480000


In [8]:
import pandas as pd
import numpy as np

eBayAuctions_df

# 범주별 평균값 계산
endDay_means = eBayAuctions_df.groupby("endDay")["Competitive?"].mean()

# 병합된 그룹 생성
merged_groups = merge_categories(endDay_means, 0.02)

# 데이터프레임에 병합된 그룹 추가
eBayAuctions_df["Merged_endDay"] = eBayAuctions_df["endDay"].map(merged_groups)

merged_endDay_pivot = eBayAuctions_df.pivot_table(index='Merged_endDay', 
                                                  values='Competitive?', 
                                                  aggfunc='mean')
print(merged_endDay_pivot)
# heatmap 시각화
sns.heatmap(merged_endDay_pivot, cmap='coolwarm')
plt.show()

               Competitive?
Merged_endDay              
Fri+Wed+Sun        0.477143
Mon                0.673358
Sat                0.427350
Thu                0.603960
Tue                0.532164


---
## Duration(경매기간)
- 1
- 3
- 5
- 7
- 10

In [9]:
Duration_pivot = eBayAuctions_df.pivot_table(index='Duration', values='Competitive?', aggfunc='mean')
print(Duration_pivot)
# heatmap 시각화
sns.heatmap(Duration_pivot, cmap='coolwarm')
plt.show()

          Competitive?
Duration              
1             0.521739
3             0.450704
5             0.686695
7             0.489142
10            0.544554


In [10]:
def circle_labeling(data, colum_name):
    data[colum_name] = pd.Categorical(data[colum_name])
    data[colum_name] = data[colum_name].cat.codes
    category_num = len(data[colum_name].unique())
    embedding = np.array(data[colum_name].values)
    #for unique_num in category:
    embedding = 2 * np.pi * embedding / category_num 
    dim1 = np.sin(embedding)
    dim2 = np.cos(embedding)
    data[colum_name + "emb_dim_1"] = dim1
    data[colum_name + "emb_dim_2"] = dim2
    return data

In [11]:
'''
범주형 예측 변수들에 대해서 더비 변수 생성
범주형 변수: Category, currency, endDay, Duration
'''

#=========== LabelEncoder  / circle_labeling ===========#

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()

preprocessed_df = eBayAuctions_df.copy()
preprocessed_df['Category'] = le.fit_transform(preprocessed_df['Category'])
preprocessed_df['currency'] = le.fit_transform(eBayAuctions_df['currency'])
preprocessed_df['endDay'] = le.fit_transform(eBayAuctions_df['endDay'])
# preprocessed_df['Duration'] = le.fit_transform(eBayAuctions_df['Duration'])
preprocessed_df = circle_labeling(preprocessed_df, 'endDay')
preprocessed_df['MergedCategory'] = le.fit_transform(eBayAuctions_df['MergedCategory'])
preprocessed_df['Merged_endDay'] = le.fit_transform(eBayAuctions_df['Merged_endDay'])
preprocessed_df



#==================== get_dummies ====================#

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from dmba.metric import AIC_score

# convert to categorical
eBayAuctions_df.Category = eBayAuctions_df.Category.astype('category')
eBayAuctions_df.currency = eBayAuctions_df.currency.astype('category')
eBayAuctions_df.Duration = eBayAuctions_df.Duration.astype('category')
eBayAuctions_df.endDay = eBayAuctions_df.endDay.astype('category')

predictors = ['Category', 'currency', 'Duration', 'endDay', 'ClosePrice', 'OpenPrice']
outcome = 'Competitive?'

X = pd.get_dummies(eBayAuctions_df[predictors])
y = eBayAuctions_df[outcome]

# split into training and validation
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4,
                                                      random_state=1)

logit_full = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_full.fit(train_X, train_y)

pd.set_option('display.width', 95)
pd.set_option('display.precision',3)
pd.set_option('display.max_columns', 50)
print('intercept ', logit_full.intercept_[0])

print(pd.DataFrame({'coeff': logit_full.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_full.predict(valid_X), df=len(train_X.columns) + 1))
pd.reset_option('display.width')
pd.reset_option('display.precision')
pd.reset_option('display.max_columns')


intercept  -0.14641848602841676
       ClosePrice  OpenPrice  Category_Antique/Art/Craft  Category_Automotive  \
coeff       0.091     -0.108                       0.296               -0.393   

       Category_Books  Category_Business/Industrial  Category_Clothing/Accessories  \
coeff           0.484                         1.437                         -1.134   

       Category_Coins/Stamps  Category_Collectibles  Category_Computer  Category_Electronics  \
coeff                 -1.923                  0.123              0.071                 0.637   

       Category_EverythingElse  Category_Health/Beauty  Category_Home/Garden  \
coeff                   -1.299                  -1.631                 0.151   

       Category_Jewelry  Category_Music/Movie/Game  Category_Photography  \
coeff            -0.526                      0.316                 2.997   

       Category_Pottery/Glass  Category_SportingGoods  Category_Toys/Hobbies  currency_EUR  \
coeff                   0.006  

- 데이터를 학습 데이터셋(60%)과 검증 데이터셋(40%)으로 분할한다. 컷오프 값을 0.5로 하여 모든 예측 변수들을 사용한 로지스틱 모델을 실행하시오.

In [ ]:
from scipy import stats
def get_statistics(model, intercept=0, coef=[], train_df=None, target_df=None):
    print("Coefficient :", coef)
    print("Intercept :", intercept)
    
    params = np.append(intercept, coef)
    print("Params:", params)

    #prediction = model.predict(train_df.values.reshape(-1, 1))        # 단변량
    prediction = model.predict(train_df.values)                     # 다변량

    if len(prediction.shape) == 1:
        prediction = np.expand_dims(prediction, axis=1)
    print(train_df.columns)

    new_trainset = pd.DataFrame({"Constant": np.ones(len(train_df.values))}).join(pd.DataFrame(train_df.values))
    print(new_trainset)

    from sklearn.metrics import mean_squared_error
    MSE = mean_squared_error(prediction, target_df.values)
    print("MSE :", MSE)

    variance = MSE * (np.linalg.inv(np.dot(new_trainset.T, new_trainset)).diagonal())       # MSE = (1, ) & else = (n, ) 가 나와야 함.

    std_error = np.sqrt(variance)
    t_values = params / std_error
    p_values = [2 * (1 - stats.t.cdf(np.abs(i), (len(new_trainset) - len(new_trainset.columns) - 1))) for i in t_values]

    std_error = np.round(std_error, 3)
    t_values = np.round(t_values, 3)
    p_values = np.round(p_values, 3)
    params = np.round(params, 4)

    statistics = pd.DataFrame()
    statistics["Coefficients"], statistics["Standard Errors"], statistics["t -values"], statistics["p-values"] = [params, std_error, t_values, p_values]
    statistics["variables"] = ["const"] + list(train_df.columns)
    
    return statistics

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from dmba.metric import AIC_score
from dmba import classificationSummary

original_df = pd.read_csv('../data/eBayAuctions.csv')
y = original_df['Competitive?']
X = preprocessed_df.drop(['Competitive?'], axis=1)
#X = X.drop(['Group'], axis=1)
X = X.drop(['MergedCategory'], axis=1)
X = X.drop(['Merged_endDay'], axis=1)
print(X.columns)

# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# fit a logistic regression (set penalty=l2 and C=1e42 to avoid regularization)
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(train_X, train_y)

logit_reg_pred = logit_reg.predict(valid_X)
logit_reg_proba = logit_reg.predict_proba(valid_X)
logit_result = pd.DataFrame({'actual': valid_y,
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })

print('intercept ', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_reg.predict(valid_X), df = len(train_X.columns) + 1))

logit_reg_pred = logit_reg.predict(valid_X)
# logit_reg_proba = logit_reg.predict_proba(valid_X)
cutoff = 0.5  # 원하는 cutoff 값 설정
logit_reg_pred = (logit_reg.predict_proba(valid_X)[:, 1] >= cutoff).astype(int)

logit_result = pd.DataFrame({'actual': valid_y,
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })

print(logit_result)

classificationSummary(train_y, logit_reg.predict(train_X))
classificationSummary(valid_y, logit_reg.predict(valid_X))

Index(['Category', 'currency', 'sellerRating', 'Duration', 'endDay',
       'ClosePrice', 'OpenPrice', 'endDayemb_dim_1', 'endDayemb_dim_2'],
      dtype='object')
intercept  -0.3476161229936271
       Category  currency  sellerRating  Duration    endDay  ClosePrice  \
coeff  0.036485  0.277425     -0.000041 -0.004843 -0.167303    0.088114   

       OpenPrice  endDayemb_dim_1  endDayemb_dim_2  
coeff  -0.109879        -0.335092        -0.056418  

AIC 1090.6977790579342
      actual      p(0)      p(1)  predicted
1287       0  0.739944  0.260056          0
1017       1  0.162643  0.837357          1
1047       0  0.607388  0.392612          0
108        1  0.343146  0.656854          1
1084       1  0.508268  0.491732          0
...      ...       ...       ...        ...
1118       1  0.564449  0.435551          0
395        0  0.691286  0.308714          0
1564       0  0.718164  0.281836          0
1698       1  0.030811  0.969189          1
1665       1  0.039036  0.960964        

In [14]:
pice = get_statistics(logit_reg, logit_reg.intercept_[0], logit_reg.coef_[0], train_X, train_y)
print(pice)

Coefficient : [ 3.64849641e-02  2.77425263e-01 -4.11504602e-05 -4.84330473e-03
 -1.67303121e-01  8.81138666e-02 -1.09879203e-01 -3.35091871e-01
 -5.64184230e-02]
Intercept : -0.3476161229936271
Params: [-3.47616123e-01  3.64849641e-02  2.77425263e-01 -4.11504602e-05
 -4.84330473e-03 -1.67303121e-01  8.81138666e-02 -1.09879203e-01
 -3.35091871e-01 -5.64184230e-02]
Index(['Category', 'currency', 'sellerRating', 'Duration', 'endDay',
       'ClosePrice', 'OpenPrice', 'endDayemb_dim_1', 'endDayemb_dim_2'],
      dtype='object')
      Constant     0    1       2     3    4       5      6         7  \
0          1.0   6.0  0.0   578.0  10.0  1.0    4.93   2.45  0.781831   
1          1.0   0.0  2.0  2349.0   7.0  0.0    5.61   3.60  0.000000   
2          1.0   6.0  0.0   884.0  10.0  1.0    2.45   2.45  0.781831   
3          1.0   0.0  2.0  2349.0   7.0  0.0    5.50   3.60  0.000000   
4          1.0  13.0  0.0   104.0   7.0  1.0    3.07   1.23  0.781831   
...        ...   ...  ...     ..

/home/minseo/anaconda3/envs/dvg/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from dmba.metric import AIC_score

original_df = pd.read_csv('../data/eBayAuctions.csv')
y = original_df['Competitive?']
X = preprocessed_df.drop(['Competitive?'], axis=1)
#X = X.drop(['Group'], axis=1)
X = X.drop(['Category'], axis=1)
X = X.drop(['endDay'], axis=1)
print(X.columns)


# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# fit a logistic regression (set penalty=l2 and C=1e42 to avoid regularization)
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(train_X, train_y)

logit_reg_pred = logit_reg.predict(valid_X)
logit_reg_proba = logit_reg.predict_proba(valid_X)
logit_result = pd.DataFrame({'actual': valid_y,
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })

print('intercept ', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_reg.predict(valid_X), df = len(train_X.columns) + 1))

logit_reg_pred = logit_reg.predict(valid_X)
# logit_reg_proba = logit_reg.predict_proba(valid_X)
cutoff = 0.5  # 원하는 cutoff 값 설정
logit_reg_pred = (logit_reg.predict_proba(valid_X)[:, 1] >= cutoff).astype(int)

logit_result = pd.DataFrame({'actual': valid_y,
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })

print(logit_result)

classificationSummary(train_y, logit_reg.predict(train_X))
classificationSummary(valid_y, logit_reg.predict(valid_X))

Index(['currency', 'sellerRating', 'Duration', 'ClosePrice', 'OpenPrice',
       'MergedCategory', 'Merged_endDay', 'endDayemb_dim_1',
       'endDayemb_dim_2'],
      dtype='object')
intercept  -0.08510583190724684
       currency  sellerRating  Duration  ClosePrice  OpenPrice  \
coeff  0.197744     -0.000031 -0.053684    0.087004  -0.107128   

       MergedCategory  Merged_endDay  endDayemb_dim_1  endDayemb_dim_2  
coeff       -0.076706       0.089572        -0.017426         0.200893  

AIC 1007.0784222223199
      actual      p(0)      p(1)  predicted
1287       0  0.693263  0.306737          0
1017       1  0.216881  0.783119          1
1047       0  0.587828  0.412172          0
108        1  0.326313  0.673687          1
1084       1  0.371907  0.628093          1
...      ...       ...       ...        ...
1118       1  0.544907  0.455093          0
395        0  0.738270  0.261730          0
1564       0  0.685283  0.314717          0
1698       1  0.017695  0.982305         

- 경매가 경쟁적인 경매인지 아닌지를 경매 시작 시점에서 예측하길 원한다면 경매 종가에 대한 정보를 사용할 수 없다. 경매 종가를 제외한 모든 예측변수를 이용해 앞에서와 같이 로지스틱 모델을 실행하시오. 이러한 모델은 예측 정확도 관점에서 모든 예측변수를 사용한 모델과 어떻게 비교되는가?

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from dmba.metric import AIC_score

original_df = pd.read_csv('../data/eBayAuctions.csv')
y = original_df['Competitive?']
X = preprocessed_df.drop(['Competitive?'], axis=1)
#X = X.drop(['Group'], axis=1)
X = X.drop(['MergedCategory'], axis=1)
X = X.drop(['Merged_endDay'], axis=1)
X = X.drop(['ClosePrice'], axis=1)
print(X.columns)

# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# fit a logistic regression (set penalty=l2 and C=1e42 to avoid regularization)
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(train_X, train_y)

logit_reg_pred = logit_reg.predict(valid_X)
logit_reg_proba = logit_reg.predict_proba(valid_X)
logit_result = pd.DataFrame({'actual': valid_y,
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })

print('intercept ', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_reg.predict(valid_X), df = len(train_X.columns) + 1))

logit_reg_pred = logit_reg.predict(valid_X)
# logit_reg_proba = logit_reg.predict_proba(valid_X)
cutoff = 0.5  # 원하는 cutoff 값 설정
logit_reg_pred = (logit_reg.predict_proba(valid_X)[:, 1] >= cutoff).astype(int)

logit_result = pd.DataFrame({'actual': valid_y,
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })

print(logit_result)

classificationSummary(train_y, logit_reg.predict(train_X))
classificationSummary(valid_y, logit_reg.predict(valid_X))

Index(['Category', 'currency', 'sellerRating', 'Duration', 'endDay',
       'OpenPrice', 'endDayemb_dim_1', 'endDayemb_dim_2'],
      dtype='object')
intercept  0.12925897893382807
       Category  currency  sellerRating  Duration    endDay  OpenPrice  \
coeff  0.035867 -0.013985     -0.000014  -0.03587  0.034044  -0.009212   

       endDayemb_dim_1  endDayemb_dim_2  
coeff          0.03654         0.283751  

AIC 1606.4069232873217
      actual      p(0)      p(1)  predicted
1287       0  0.587517  0.412483          0
1017       1  0.410518  0.589482          1
1047       0  0.524054  0.475946          0
108        1  0.358734  0.641266          1
1084       1  0.527029  0.472971          0
...      ...       ...       ...        ...
1118       1  0.524054  0.475946          0
395        0  0.501375  0.498625          0
1564       0  0.425609  0.574391          1
1698       1  0.505367  0.494633          0
1665       1  0.367488  0.632512          1

[789 rows x 4 columns]
Confusion 

- 경매 종가에 대한 회귀 계수의 의미를 해석하시오. 경매 종가는 실질적인 의미가 있는가? 경매 종가를 이용해 경쟁적인 경매를 예측하는 것이 통계적으로 의미가 있는가? (유의수준 10% 사용)

In [17]:
pice = get_statistics(logit_reg, logit_reg.intercept_[0], logit_reg.coef_[0], train_X, train_y)
print(pice)

Coefficient : [ 3.58671580e-02 -1.39847972e-02 -1.44006653e-05 -3.58698472e-02
  3.40439073e-02 -9.21192460e-03  3.65403437e-02  2.83750633e-01]
Intercept : 0.12925897893382807
Params: [ 1.29258979e-01  3.58671580e-02 -1.39847972e-02 -1.44006653e-05
 -3.58698472e-02  3.40439073e-02 -9.21192460e-03  3.65403437e-02
  2.83750633e-01]
Index(['Category', 'currency', 'sellerRating', 'Duration', 'endDay',
       'OpenPrice', 'endDayemb_dim_1', 'endDayemb_dim_2'],
      dtype='object')
      Constant     0    1       2     3    4      5         6         7
0          1.0   6.0  0.0   578.0  10.0  1.0   2.45  0.781831  0.623490
1          1.0   0.0  2.0  2349.0   7.0  0.0   3.60  0.000000  1.000000
2          1.0   6.0  0.0   884.0  10.0  1.0   2.45  0.781831  0.623490
3          1.0   0.0  2.0  2349.0   7.0  0.0   3.60  0.000000  1.000000
4          1.0  13.0  0.0   104.0   7.0  1.0   1.23  0.781831  0.623490
...        ...   ...  ...     ...   ...  ...    ...       ...       ...
1178       1.

/home/minseo/anaconda3/envs/dvg/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


# Archive - Previous e,f solution

- [6장 다중선형회귀]에서 설명한 대로 단계적 회귀(stepwise regression)를 사용해 학습 데이터셋에 가장 잘 적합된(정확도가 가장 높은) 모델을 찾으시오. 어떤 예측변수가 사용되는가?

In [18]:
import statsmodels.api as sm
import time
import warnings
warnings.filterwarnings("ignore")
def stepwise_selection(data_x, data_y):
    # 초기화
    startpoint = time.time()
    selected_features = []
    best_aic = float("inf")  # 초기 AIC를 무한대로 설정
    loopCount = 0

    while True:
        added = None
        removed = None
        # Forward Selection
        for feature in data_x.columns:
            if feature not in selected_features:
                temp_features = selected_features + [feature]
                
                model = sm.Logit(data_y, sm.add_constant(data_x[temp_features])).fit(method = 'bfgs')
                aic = model.aic
                if aic < best_aic:
                    best_aic = aic
                    added = feature
                    
                    
        # Backward Elimination
        if len(selected_features) > 0:
            for feature in selected_features:
                temp_features = [f for f in selected_features if f != feature]
                model = sm.Logit(data_y, sm.add_constant(data_x[temp_features])).fit(method = 'bfgs')
                aic = model.aic
                if aic < best_aic:
                    best_aic = aic
                    removed = feature
        if added is not None:
            selected_features.append(added)
        elif removed is not None:
            selected_features.remove(removed)
        else:
            break
        time.sleep(1)
        loopCount += 1
    #print("Best selected features:", selected_features)
    #print("Best AIC:", best_aic)

    return selected_features

In [19]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import statsmodels.api as sm

original_df = pd.read_csv('../data/eBayAuctions.csv')
y = original_df['Competitive?']
X = preprocessed_df.drop(['Competitive?'], axis=1)
X = X.drop(['Category'], axis=1)
X = X.drop(['endDay'], axis=1)
X = X.drop(['ClosePrice'], axis=1)
print(X.columns)

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)
# partition data
selected_features = stepwise_selection(train_X, train_y)
print(selected_features)

# Fit Final Model
final_model = LogisticRegression()
final_model.fit(train_X[selected_features], train_y)
print("Final Model Coefficients:", final_model.coef_)

Index(['currency', 'sellerRating', 'Duration', 'OpenPrice', 'MergedCategory',
       'Merged_endDay', 'endDayemb_dim_1', 'endDayemb_dim_2'],
      dtype='object')
Optimization terminated successfully.
         Current function value: 0.690917
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10
         Current function value: 0.693147
         Iterations: 0
         Function evaluations: 13
         Gradient evaluations: 2
Optimization terminated successfully.
         Current function value: 0.690785
         Iterations: 11
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.682771
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.690244
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 5
Optimization terminate

- 검증 데이터셋에서 가장 정확도가 높은 모델을 찾기 위해 단계적 회귀(stepwise regression)를 사용하시오. 어떤 예측변수가 사용되는가?

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import statsmodels.api as sm

original_df = pd.read_csv('../data/eBayAuctions.csv')
y = original_df['Competitive?']
X = preprocessed_df.drop(['Competitive?'], axis=1)
X = X.drop(['Category'], axis=1)
X = X.drop(['endDay'], axis=1)
X = X.drop(['ClosePrice'], axis=1)
print(X.columns)

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)
# partition data
selected_features = stepwise_selection(train_X, train_y)
print(selected_features)

# Fit Final Model
final_model = LogisticRegression()
final_model.fit(valid_X[selected_features], valid_y)
print("Final Model Coefficients:", final_model.coef_)

Index(['currency', 'sellerRating', 'Duration', 'OpenPrice', 'MergedCategory',
       'Merged_endDay', 'endDayemb_dim_1', 'endDayemb_dim_2'],
      dtype='object')
Optimization terminated successfully.
         Current function value: 0.690917
         Iterations: 9
         Function evaluations: 10
         Gradient evaluations: 10
         Current function value: 0.693147
         Iterations: 0
         Function evaluations: 13
         Gradient evaluations: 2
Optimization terminated successfully.
         Current function value: 0.690785
         Iterations: 11
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.682771
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.690244
         Iterations: 4
         Function evaluations: 5
         Gradient evaluations: 5
Optimization terminate

# Part II - Jaeheon

### e/f. 단계적 회귀 (stepwise_selection)

In [21]:
'''
stepwise_selection with the example(Chapter_06_linear_regression.ipynb)
'''

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from dmba import stepwise_selection

predictors = ['MergedCategory', 'currency', 'Duration', 'Merged_endDay', 'ClosePrice', 'OpenPrice', 'sellerRating']
outcome = 'Competitive?'

X = pd.get_dummies(eBayAuctions_df[predictors])
y = eBayAuctions_df[outcome]

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

def train_model(variables):
    if len(variables) == 0:
        return None
    model = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
    model.fit(train_X[variables], train_y)
    return model

def score_model_w_train(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [int(train_y.mean())] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

def score_model_w_valid(model, variables):
    if len(variables) == 0:
        return AIC_score(valid_y, [int(valid_y.mean())] * len(valid_y), model, df=1)
    return AIC_score(valid_y, model.predict(valid_X[variables]), model)


pd.set_option('display.width', 95)
pd.set_option('display.precision',3)
pd.set_option('display.max_columns', 50)

print('\n', '='*25, 'Select the best variables with a train AIC score', '='*25)
best_model, best_variables = stepwise_selection(X.columns, train_model, score_model_w_train, verbose=True)
print(pd.DataFrame({'coeff': best_model.coef_[0]}, index=best_variables).transpose())
print('AIC', AIC_score(valid_y, best_model.predict(valid_X[best_variables]), df = len(best_variables) + 1))

print('\n', '='*25, 'Select the best variables with a valid AIC score', '='*25)
best_model, best_variables = stepwise_selection(X.columns, train_model, score_model_w_valid, verbose=True)
print(pd.DataFrame({'coeff': best_model.coef_[0]}, index=best_variables).transpose())
print('AIC', AIC_score(valid_y, best_model.predict(valid_X[best_variables]), df = len(best_variables) + 1))

pd.reset_option('display.width')
pd.reset_option('display.precision')
pd.reset_option('display.max_columns')


 ========================= Select the best variables with a train AIC score =========================
Variables: ClosePrice, OpenPrice, sellerRating, MergedCategory_Books+Clothing/Accessories+Toys/Hobbies+Antique/Art/Craft+Collectibles+Music/Movie/Game, MergedCategory_Coins/Stamps, MergedCategory_Electronics+Photography, MergedCategory_EverythingElse, MergedCategory_Health/Beauty, MergedCategory_Home/Garden+Computer+Business/Industrial, MergedCategory_Pottery/Glass+Automotive+Jewelry, MergedCategory_SportingGoods, currency_EUR, currency_GBP, currency_US, Duration_1, Duration_3, Duration_5, Duration_7, Duration_10, Merged_endDay_Fri+Wed+Sun, Merged_endDay_Mon, Merged_endDay_Sat, Merged_endDay_Thu, Merged_endDay_Tue
Start: score=2615.81, constant
Step: score=2245.77, add ClosePrice
Step: score=1641.46, add OpenPrice
Step: score=1481.04, add MergedCategory_Books+Clothing/Accessories+Toys/Hobbies+Antique/Art/Craft+Collectibles+Music/Movie/Game
Step: score=1334.94, add Merged_endDay_Fri+We

In [22]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

predictors = ['MergedCategory', 'Merged_endDay']
outcome = 'Competitive?'

X = eBayAuctions_df[predictors]
y = eBayAuctions_df[outcome]

les = [LabelEncoder(), LabelEncoder()]
for i in range(len(predictors)):
    X[predictors[i]] = les[i].fit_transform(X[predictors[i]])

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

train_df = pd.concat([train_X, train_y], axis=1)
valid_df = pd.concat([valid_X, valid_y], axis=1)

plt.figure(figsize=(10, 5))
figs, axs = plt.subplots(1, 2, figsize=(10, 5))



for i in range(len(predictors)):
    groupAverage_valid = valid_df.groupby(predictors[i])[outcome].mean()
    groupAverage_train = train_df.groupby(predictors[i])[outcome].mean()

    width = 0.4

    x = range(len(groupAverage_valid))

    axs[i].bar([i + width / 2 for i in x], groupAverage_train, width=width, color='tab:blue', label='Train')
    axs[i].bar([i - width / 2 for i in x], groupAverage_valid, width=width, color='tab:orange', label='Valid')

    max_label_length = 10
    truncated_labels = [label[:max_label_length] + "..." if len(label) > max_label_length else label
                        for label in les[i].inverse_transform(range(len(les[i].classes_)))]

    if i == 0:
        axs[i].set_ylabel('Average Competitive?')
    axs[i].set_xticks(x)
    axs[i].set_xticklabels(truncated_labels, rotation=45, ha='right')

    axs[i].legend()

plt.show()

### - i. 학습 데이터셋에 L1 페널티가 있는 벌점화(regularized) 로지스틱 회귀를 사용한다. 이 결과(예측변수와 분류 성능)를, 가장 잘 적합한 모델 및 가장 잘 예측한 모델의 결과와 비교하시오.

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from dmba.metric import AIC_score
from dmba import classificationSummary, gainsChart

predictors = ['MergedCategory', 'currency', 'Duration', 'Merged_endDay', 'ClosePrice', 'OpenPrice', 'sellerRating']
outcome = 'Competitive?'

X = pd.get_dummies(eBayAuctions_df[predictors])[best_variables]
y = eBayAuctions_df[outcome]

# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)


########### best model ###############
logit_best = best_model

logit_best_proba = logit_best.predict_proba(valid_X)
best_result = pd.DataFrame({'actual': valid_y,
                            'p(0)': [p[0] for p in logit_best_proba],
                            'p(1)': [p[1] for p in logit_best_proba],
                            'predicted': logit_best.predict(valid_X),
                          })
best_result = best_result.sort_values(by=['p(1)'], ascending=False)


########### l2-regularized model ###############
logit_best_l2 = LogisticRegression(penalty="l2", solver='liblinear')
logit_best_l2.fit(train_X, train_y)

logit_best_l2_proba = logit_best_l2.predict_proba(valid_X)
best_l2_result = pd.DataFrame({'actual': valid_y,
                            'p(0)': [p[0] for p in logit_best_l2_proba],
                            'p(1)': [p[1] for p in logit_best_l2_proba],
                            'predicted': logit_best_l2.predict(valid_X),
                          })
best_l2_result = best_l2_result.sort_values(by=['p(1)'], ascending=False)



########### l1-regularized model ###############
logit_l1 = LogisticRegression(penalty="l1", solver='liblinear')
logit_l1.fit(train_X, train_y)

logit_l1_proba = logit_l1.predict_proba(valid_X)
l1_result = pd.DataFrame({'actual': valid_y,
                            'p(0)': [p[0] for p in logit_l1_proba],
                            'p(1)': [p[1] for p in logit_l1_proba],
                            'predicted': logit_l1.predict(valid_X),
                          })
l1_result = l1_result.sort_values(by=['p(1)'], ascending=False)


pd.set_option('display.width', 95)
pd.set_option('display.precision',3)
pd.set_option('display.max_columns', 50)

print('='*20, 'Best model', '='*25)
print(pd.DataFrame({'coeff': logit_best.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_best.predict(valid_X), df=len(train_X.columns) + 1))
classificationSummary(best_result['actual'], best_result['predicted'])

# print('\n', '='*17, 'L2-Regularized model', '='*17)
# print(pd.DataFrame({'coeff': logit_best_l2.coef_[0]}, index=X.columns).transpose())
# print()
# print('AIC', AIC_score(valid_y, logit_best_l2.predict(valid_X), df=len(train_X.columns) + 1))
# classificationSummary(best_l2_result['actual'], best_l2_result['predicted'])

print('\n', '='*17, 'L1-Regularized model', '='*17)
print(pd.DataFrame({'coeff': logit_l1.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_l1.predict(valid_X), df=len(train_X.columns) + 1))
classificationSummary(l1_result['actual'], l1_result['predicted'])


pd.reset_option('display.width')
pd.reset_option('display.precision')
pd.reset_option('display.max_columns')

ax = gainsChart(best_result.actual, label='Best model', color='C1', figsize=[5, 5])
# ax = gainsChart(best_l2_result.actual, label='L1-Regularized model', color='C1', ax=ax)
ax = gainsChart(l1_result.actual, label='L1-Regularized model', color='C0', ax=ax)
ax.legend()

plt.show()

==================== Best model =========================
       ClosePrice  OpenPrice  \
coeff       0.075     -0.091   

       MergedCategory_Books+Clothing/Accessories+Toys/Hobbies+Antique/Art/Craft+Collectibles+Music/Movie/Game  \
coeff                                              0.819                                                        

       MergedCategory_Home/Garden+Computer+Business/Industrial  MergedCategory_Health/Beauty  \
coeff                                              0.851                              -1.057   

       MergedCategory_Electronics+Photography  Merged_endDay_Sat  \
coeff                                   1.245             -0.507   

       MergedCategory_SportingGoods  
coeff                         0.575  

AIC 836.335059659033
Confusion Matrix (Accuracy 0.8352)

       Prediction
Actual   0   1
     0 332  21
     1 109 327

 ================= L1-Regularized model =================
       ClosePrice  OpenPrice  \
coeff       0.076     -0.092   


### - j. 모델 분석의 주목적이 정확한 분류라고 한다면, 어떤 컷오프 값이 사용되어야 하는가?

In [24]:
from sklearn.metrics import f1_score, accuracy_score

max_f1 = 0
max_acc = 0
cutoffs = []
f1s = []
accs = []
for cutoff in np.arange(0.0, 1.0, 0.001):
    cutoffs.append(cutoff)
    logit_best_pred = (logit_best.predict_proba(valid_X)[:, 1] >= cutoff).astype(int)
    f1 = f1_score(valid_y, logit_best_pred)
    f1s.append(f1)
    if f1 > max_f1:
        max_f1 = f1
        max_f1_cutoff = cutoff

    acc = accuracy_score(valid_y, logit_best_pred)
    accs.append(acc)
    if acc > max_acc:
        max_acc = acc
        max_acc_cutoff = cutoff

plt.figure(figsize=(4, 3))

plt.plot(cutoffs, f1s, label='F1-score', color='tab:blue')
plt.plot(cutoffs, accs, label='Accuracy', color='tab:orange')

plt.vlines(max_acc_cutoff, 0, 1, linestyles='--', colors='gray')
plt.text(max_f1_cutoff+0.01, 0.05, str(max_f1_cutoff), color='gray')
plt.text(max_f1_cutoff+0.01, max_f1+0.06, str(max_f1), color='tab:blue')
plt.text(max_acc_cutoff+0.01, max_acc, str(max_acc), color='tab:orange')

plt.legend()
plt.tight_layout()
plt.show()

### - k. 이 데이터를 바탕으로 판매자가 정한 경매 조건(경매 기간, 시작가, 경매 마감 요일, 화폐 단위) 중 어떤 설정이 경쟁적인 판매로 이어질 가능성을 가장 높인다고 할 수 있겠는가?

In [25]:
seller_setting = ['Duration', 'OpenPrice', 'Merged_endDay', 'currency']

X = pd.get_dummies(eBayAuctions_df[seller_setting])
y = eBayAuctions_df[outcome]

# partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(train_X, train_y)

logit_reg_proba = logit_reg.predict_proba(valid_X)

pd.set_option('display.width', 95)
pd.set_option('display.precision',3)
pd.set_option('display.max_columns', 50)
print('intercept ', logit_reg.intercept_[0])

print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())
print()
print('AIC', AIC_score(valid_y, logit_reg.predict(valid_X), df=len(train_X.columns) + 1))
pd.reset_option('display.width')
pd.reset_option('display.precision')
pd.reset_option('display.max_columns')

intercept  0.37491744321690385
       OpenPrice  Duration_1  Duration_3  Duration_5  Duration_7  Duration_10  \
coeff     -0.008       0.355      -0.115       0.382      -0.198        -0.05   

       Merged_endDay_Fri+Wed+Sun  Merged_endDay_Mon  Merged_endDay_Sat  Merged_endDay_Thu  \
coeff                     -0.003               0.69             -0.053             -0.405   

       Merged_endDay_Tue  currency_EUR  currency_GBP  currency_US  
coeff              0.146        -0.311         1.099       -0.413  

AIC 1568.8570122097267
